# `08-train-models`

Now we are in a position to train different models on the `Train` dataset!

In [1]:
import numpy, pandas

from sklearn.model_selection import StratifiedKFold, GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier

from xgboost import XGBClassifier

from skops.io import dump

from misc import construct_line

pandas.options.display.max_columns=999

### Model training

Now we can perform hyperparameter tuning using crossfold validation on the `training` set, select the best model and record its performance, which as it has been measured independently on each fold, naturally has a variance.

Note that 
* the `random_seed` is fixed to ensure reproducibility
* since the `1` state is resistance we are predicting resistance
* we are optimising using `recall` to ensure the very major error (VME) rate is minimised


In [2]:
random_seed = 144
n_folds=10    
optimisation_score='recall'
scoring_metrics=['recall', 'roc_auc', 'precision']
folds = StratifiedKFold(n_splits=n_folds, random_state=random_seed, shuffle=True)
line=[]
very_major_errors = {}
major_errors = {}

Let's load the `training` arrays

In [11]:
X={}
Y={}
Z={}

X['train']={}
Y['train']={}
Z['train']={}

with open('data/ds-train.npy', 'rb') as f:
    Y['train']['input'] = numpy.load(f)
    X['train']['input'] = numpy.load(f)
    Z['train']['input'] = numpy.load(f, allow_pickle=True)

The below function performs the hyperparameter tuning on a provided model and records several performance metrics

In [12]:
def tune_model(line, model, model_name, X, Y, param_grid, folds, optimisation_score):

    # hyperparameter tuning
    grid_search = GridSearchCV( model, 
                                param_grid, 
                                cv=folds, 
                                n_jobs=-1, 
                                return_train_score=True,
                                scoring=scoring_metrics,
                                refit=optimisation_score )

    grid_search.fit(X['train']['input'], Y['train']['input'])

    cv_results = pandas.DataFrame(grid_search.cv_results_)

    # get the best model
    best_model = grid_search.best_estimator_

    cv_test_scores = []
    for i in range(10):
        value = cv_results[cv_results['rank_test_'+optimisation_score]==1]['split'+str(i)+'_test_recall']
        # try:
        cv_test_scores.append(float(value))
        # except:
        #     print(value)

    scores = []
    for score in scoring_metrics:
        mean_score = cv_results[cv_results['rank_test_'+score]==1]['mean_test_'+score].values[0]
        std_score = cv_results[cv_results['rank_test_'+score]==1]['std_test_'+score].values[0]
        scores.append(100*mean_score)
        scores.append(100*std_score)
    Y['train']['predicted'] = grid_search.predict(X['train']['input'])
    Y['train']['scores'] = grid_search.predict_proba(X['train']['input'])[:,1]

    row = construct_line(model_name , 'train', scores, Y['train'], grid_search.best_params_)
    line.append(row)

    return(line, best_model, cv_test_scores)

### 1. Logistic regression

In [5]:
model_logistic_regression = LogisticRegression(random_state=random_seed, class_weight='balanced')

param_grid = [ 
                {   'penalty': ['l1', 'l2'],
                    'solver'  : ['newton-cg', 'lbfgs', 'liblinear'], 
                    'C': numpy.logspace(-3,3,7)
                } 
            ]

line, best_LR, cv_results_LR = tune_model(line, model_logistic_regression, "LR", X, Y, param_grid, folds, optimisation_score)

dump(best_LR, 'models/lr.skops')


/Users/fowler/Library/Python/3.11/lib/python/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/fowler/Library/Python/3.11/lib/python/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/fowler/Library/Python/3.11/lib/python/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/fowler/Library/Python/3.11/lib/python/site-packages/sklearn/met

### 2. Multilayer perception classifier

In [6]:
model_nn = MLPClassifier(random_state=random_seed)

param_grid = [ 
        {   'solver': ['lbfgs'],
            'max_iter': [400, 800],
            'alpha':10.0 ** -numpy.arange(1, 7),
            'hidden_layer_sizes':[(100,), (20,10,5), (10,5), (20,10), (100,50,10)] ,
            'activation':['relu','logistic','tanh']} 
    ]

line, best_NN, cv_results_NN = tune_model(line, model_nn, "NN", X, Y, param_grid, folds, optimisation_score)

dump(best_NN, 'models/nn.skops')

/Users/fowler/Library/Python/3.11/lib/python/site-packages/sklearn/neural_network/_multilayer_perceptron.py:546: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)
/Users/fowler/Library/Python/3.11/lib/python/site-packages/sklearn/neural_network/_multilayer_perceptron.py:546: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)
/Users/fowler/Library/Python/3.11/lib/python/site-packages/sklearn/neural_network/_multilayer_perceptron.py:546: ConvergenceWarning: lbfgs faile

### 3. XGBoost Classifier

In [13]:
model_xgb = XGBClassifier()

param_grid = [ 
        {   'n_estimators': [25, 50, 100, 150, 200],
            'subsample': [0.6, 0.7, 0.8, 0.9, 1],
            'max_depth':[2,4,5,6,8],
            'min_child_weight': [0, 0.1, 1, 10],
            'learning_rate':[0.005, 0.01, 0.05, 0.075, 0.1, 0.125, 0.1] } 
    ]

line, best_XB, cv_results_XB = tune_model(line, model_xgb, "XB", X, Y, param_grid, folds, optimisation_score)

dump(best_XB, 'models/xb.skops')


TypeError: cannot convert the series to <class 'float'>

Finally let's store everything in a Pandas dataframe to make it easy to query and draw figures etc

In [9]:
training_results = pandas.DataFrame(line, columns=['model', 'dataset', 'sensitivity_mean', 'sensitivity_std', 'specificity_mean', 'specificity_std' ,'roc_auc_mean', 'roc_auc_std','TN','FP','FN','TP', 'model_parameters'])

# calculate the diagnostic odds ration
training_results['diagnostic_odds_ratio_mean'] = (training_results['TN']*training_results['TP'])/(training_results['FN']*training_results['FP'])
training_results['diagnostic_odds_ratio_std'] = 0

# save to disc as a CSV
training_results.to_csv('results-training.csv', index=False)

training_results

,model,dataset,sensitivity_mean,sensitivity_std,specificity_mean,specificity_std,roc_auc_mean,roc_auc_std,TN,FP,FN,TP,model_parameters,diagnostic_odds_ratio_mean,diagnostic_odds_ratio_std
0,LR,train,78.766667,7.969804,84.635931,4.924239,82.734349,6.631325,178,40,47,199,"{""C"": 0.1, ""penalty"": ""l2"", ""solver"": ""libline...",18.841489,0
1,NN,train,77.700000,9.539043,83.269372,3.173211,78.364529,5.500773,213,5,1,245,"{""activation"": ""relu"", ""alpha"": 1e-06, ""hidden...",10437.000000,0
2,XB,train,93.916667,3.717265,85.622186,4.544960,83.832217,6.613047,98,120,15,231,"{""learning_rate"": 0.005, ""max_depth"": 4, ""min_...",12.576667,0
